## data 준비 파트

In [1]:
import sys
import os
import gc

import warnings
warnings.filterwarnings('ignore')

import random
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import duckdb

import lightgbm as lgb
import xgboost as xgb
import catboost as cat

import shap

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
from plotnine import * 
from mizani.breaks import date_breaks
from mizani.formatters import date_format

In [2]:
BASEPATH = '/Users/djroz/Desktop/kaggle_study/rohlik-sales-v2-data/'

In [3]:
# borrowed from https://www.kaggle.com/code/samvelkoch/rohlik2-lama-v6-weighted

calendar = duckdb.query(f'''
    SELECT 
       t.* exclude(holiday_name),
       coalesce(t.holiday_name, '-') as holiday_name
    FROM 
        read_csv('{BASEPATH + 'calendar.csv'}') t
    ''').df()

from datetime import datetime
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    df['school'] = df['winter_school_holidays'] + df['school_holidays']
    return df
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = (
    pd.concat(processed_dfs)
    .sort_values('date')
    .drop(columns=['winter_school_holidays', 'school_holidays'])
    #.query('date <= "2024-06-02"')
    .reset_index(drop=True))

In [4]:
# read and join data

duckdb.query(f'''
    create or replace table sales as
    SELECT 
        s.*,
        'train' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_train.csv'}') s
    WHERE
        -- 52 rows have a null sales observation. 
        -- we just drop them.
        s.sales is not null
    UNION ALL BY NAME
    SELECT 
        s.*,
        'test' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_test.csv'}') s
    ''')


duckdb.query(f'''
    create or replace table inventory as
    SELECT 
       t.* exclude(L1_category_name_en, L2_category_name_en, L3_category_name_en, L4_category_name_en),
       L1_category_name_en as l1_cat,
       L1_category_name_en || '-' ||  L2_category_name_en as l2_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en as l3_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en || '-' ||  L3_category_name_en as l4_cat
    FROM 
        read_csv('{BASEPATH + 'inventory.csv'}') t
    ''')

duckdb.query(f'''
    create or replace table calendar as
    SELECT 
       *
    FROM 
        calendar_extended
    ''')

duckdb.query(f'''
    create or replace table test_weights as
    SELECT 
       *
    FROM 
        read_csv('{BASEPATH + 'test_weights.csv'}') t
    ''')

duckdb.query(f'''
    create or replace view all_data as
    SELECT 
        s.*,
        c.* exclude(warehouse, date),
        i.* exclude(warehouse, unique_id),
        w.* exclude(unique_id),
        -- group into fortnights
        -- The whole of test is a single fold
        date_diff('day', '2020-07-20', s.date) // 14 as dt_fold,
        date_diff('day', '2020-07-20', s.date) % 14  as dt_fold_day,
    FROM 
        sales s
        join calendar c on c.warehouse = s.warehouse and c.date = s.date
        join inventory i on i.warehouse = s.warehouse and i.unique_id = s.unique_id
        join test_weights w on w.unique_id = s.unique_id
''')

## Fit model 파트

In [5]:
# add groupings for previous weeks:

groupings = ['l1_cat', 'l2_cat', 'l3_cat', 'l4_cat', 'name']
for grouping in groupings:
    # without warehouse
    duckdb.query(f'''
        create or replace table {grouping}_hist as
        select dt_fold, {grouping}, mean(sales) as sales
        from all_data
        where dt_fold > 0
        group by dt_fold, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')
    # with warehouse
    duckdb.query(f'''
        create or replace table {grouping}_whist as
        select dt_fold, warehouse, {grouping}, mean(sales) as sales, stddev(sales) as sales_sd,
        from all_data
        where dt_fold > 0
        group by dt_fold, warehouse, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')

# same day:
duckdb.query(f'''
    create or replace table name_whist_day as
    select dt_fold, dt_fold_day, warehouse, name, sum(sales) as sales
    from all_data
    where dt_fold > 0
    group by dt_fold, dt_fold_day, warehouse, name
    having sum(sales) is not null
    order by dt_fold, warehouse, name''')

In [6]:
duckdb.query(f'''
    create or replace table fe_data as
    select 
        a.*,
        -- sales of similar products in previous fortnight.
        -- overall
        coalesce(l1_cat_hist.sales, 0) as l1_cat_m1,
        coalesce(l2_cat_hist.sales, 0) as l2_cat_m1,
        coalesce(l3_cat_hist.sales, 0) as l3_cat_m1,
        coalesce(l4_cat_hist.sales, 0) as l4_cat_m1,
        coalesce(name_hist.sales, 0) as name_m1,
        -- by warehouse
        coalesce(l1_cat_whist.sales, 0) as l1_cat_wm1,
        coalesce(l2_cat_whist.sales, 0) as l2_cat_wm1,
        coalesce(l3_cat_whist.sales, 0) as l3_cat_wm1,
        coalesce(l4_cat_whist.sales, 0) as l4_cat_wm1,
        -- name & warehouse last 3
        coalesce(m1.sales, 0) as name_wm1,
        coalesce(m1.sales_sd, 0) as name_sd_wm1,
        coalesce(m2.sales, 0) as name_wm2,
        coalesce(m3.sales, 0) as name_wm3,
        coalesce(m4.sales, 0) as name_wm4,
        coalesce(m5.sales, 0) as name_wm5,
        (coalesce(m1.sales, 0) * 1
         + coalesce(m2.sales, 0) * 0.8
         + coalesce(m3.sales, 0) * 0.6
         + coalesce(m4.sales, 0) * 0.3
         + coalesce(m5.sales, 0) * 0.1 ) as name_w_weighted,
        coalesce(name_whist_day.sales, 0) as day_sales_m1,
        extract(dayofweek FROM date) AS day_of_week, 
        extract(month FROM date) AS month, 
        extract(day FROM date) AS day_of_month,
        extract(year FROM date) as year,
        extract(dayofyear FROM date) as day_of_year,
        extract(week FROM date) as week_no,
    from 
        all_data a
        -- overall all warehouses:
        left join l1_cat_hist 
            on l1_cat_hist.dt_fold + 1 = a.dt_fold 
                and l1_cat_hist.l1_cat = a.l1_cat
        left join l2_cat_hist 
            on l2_cat_hist.dt_fold + 1 = a.dt_fold 
                and l2_cat_hist.l2_cat = a.l2_cat
        left join l3_cat_hist 
            on l3_cat_hist.dt_fold + 1 = a.dt_fold 
                and l3_cat_hist.l3_cat = a.l3_cat
        left join l4_cat_hist 
            on l4_cat_hist.dt_fold + 1 = a.dt_fold 
                and l4_cat_hist.l4_cat = a.l4_cat
        left join name_hist 
            on name_hist.dt_fold + 1 = a.dt_fold 
                and name_hist.name = a.name
        -- by warehouse
        left join l1_cat_whist 
            on l1_cat_whist.dt_fold + 1 = a.dt_fold 
                and l1_cat_whist.warehouse = a.warehouse 
                and l1_cat_whist.l1_cat = a.l1_cat
        left join l2_cat_whist
            on l2_cat_whist.dt_fold + 1 = a.dt_fold 
                and l2_cat_whist.warehouse = a.warehouse 
                and l2_cat_whist.l2_cat = a.l2_cat
        left join l3_cat_whist 
            on l3_cat_whist.dt_fold + 1 = a.dt_fold 
                and l3_cat_whist.warehouse = a.warehouse 
                and l3_cat_whist.l3_cat = a.l3_cat
        left join l4_cat_whist 
            on l4_cat_whist.dt_fold + 1 = a.dt_fold 
                and l4_cat_whist.warehouse = a.warehouse 
                and l4_cat_whist.l4_cat = a.l4_cat
        left join name_whist as m1 
            on m1.dt_fold + 1 = a.dt_fold 
                and m1.warehouse = a.warehouse 
                and m1.name = a.name
        left join name_whist as m2 
            on m2.dt_fold + 2 = a.dt_fold 
                and m2.warehouse = a.warehouse 
                and m2.name = a.name
        left join name_whist as m3 
            on m3.dt_fold + 3 = a.dt_fold 
                and m3.warehouse = a.warehouse 
                and m3.name = a.name
        left join name_whist as m4 
            on m4.dt_fold + 4 = a.dt_fold 
                and m4.warehouse = a.warehouse 
                and m4.name = a.name
        left join name_whist as m5 
            on m5.dt_fold + 5 = a.dt_fold 
                and m5.warehouse = a.warehouse 
                and m5.name = a.name
        left join name_whist_day 
            on name_whist_day.dt_fold + 1 = a.dt_fold
                and name_whist_day.dt_fold_day = a.dt_fold_day
                and name_whist_day.warehouse = a.warehouse 
                and name_whist_day.name = a.name
    where
        a.dt_fold > 2
''')

In [7]:
TARGET_NAME = 'sales'

cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year',
]

features = [
    'warehouse', 'total_orders',
    'sell_price_main', 'type_0_discount', 'type_1_discount',
    'type_2_discount', 'type_3_discount', 'type_4_discount',
    'type_5_discount', 'type_6_discount',  
    'holiday',
    'shops_closed', 
    'days_to_holiday',
    'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    #'l1_cat_m1', 'l2_cat_m1', 'l3_cat_m1', 
    'l4_cat_m1', 'name_m1', 
    #'l1_cat_wm1', 'l2_cat_wm1', 'l3_cat_wm1', 
    'l4_cat_wm1', 
    'name_wm1', 'name_sd_wm1',
    'name_wm2', 'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted',
    'day_sales_m1',
    'day_of_week', 'month', 'day_of_month', 'year',
    'day_of_year', 'sin_dayofyear', 'cos_dayofyear',
    'week_no',        
]

엔지니어링 전 데이터셋 초기화 부분 (quacking 노트 기본)

In [30]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')


del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 57), (3759180, 57), (190794, 57))

In [31]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


### `all_data` 확인

In [13]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()
all_data.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'availability', 'type_0_discount', 'type_1_discount',
       'type_2_discount', 'type_3_discount', 'type_4_discount',
       'type_5_discount', 'type_6_discount', 'rowtype', 'holiday',
       'shops_closed', 'holiday_name', 'days_to_holiday',
       'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
       'product_unique_id', 'name', 'l1_cat', 'l2_cat', 'l3_cat', 'l4_cat',
       'weight', 'dt_fold', 'dt_fold_day', 'l1_cat_m1', 'l2_cat_m1',
       'l3_cat_m1', 'l4_cat_m1', 'name_m1', 'l1_cat_wm1', 'l2_cat_wm1',
       'l3_cat_wm1', 'l4_cat_wm1', 'name_wm1', 'name_sd_wm1', 'name_wm2',
       'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted', 'day_sales_m1',
       'day_of_week', 'month', 'day_of_month', 'year', 'day_of_year',
       'week_no'],
      dtype='object')

In [14]:
# all_data = duckdb.query(f'''
#     select * from fe_data''').df()
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3996995 entries, 0 to 3996994
Data columns (total 55 columns):
 #   Column                Dtype         
---  ------                -----         
 0   unique_id             int64         
 1   date                  datetime64[us]
 2   warehouse             object        
 3   total_orders          float64       
 4   sales                 float64       
 5   sell_price_main       float64       
 6   availability          float64       
 7   type_0_discount       float64       
 8   type_1_discount       float64       
 9   type_2_discount       float64       
 10  type_3_discount       float64       
 11  type_4_discount       float64       
 12  type_5_discount       float64       
 13  type_6_discount       float64       
 14  rowtype               object        
 15  holiday               int64         
 16  shops_closed          int64         
 17  holiday_name          object        
 18  days_to_holiday       float64       
 19  

In [29]:
all_data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name_w_weighted,day_sales_m1,day_of_week,month,day_of_month,year,day_of_year,week_no,sin_dayofyear,cos_dayofyear
0,4605,2022-03-01,Prague_3,4683.0,22.40,76.04,1.00,0.0,0.0,0.0,...,111.546148,43.09,2,3,1,2022,60,9,0.858764,0.512371
1,4605,2022-04-20,Prague_3,4691.0,33.88,14.00,0.13,0.0,0.0,0.0,...,115.675143,48.03,3,4,20,2022,110,16,0.948362,-0.317191
2,4605,2023-01-05,Prague_3,4997.0,45.65,90.59,0.47,0.0,0.0,0.0,...,104.884786,49.51,4,1,5,2023,5,1,0.085965,0.996298
3,4605,2022-03-06,Prague_3,4538.0,28.64,80.98,1.00,0.0,0.0,0.0,...,111.546148,35.74,0,3,6,2022,65,9,0.899631,0.436651
4,4605,2022-09-18,Prague_3,4707.0,51.88,79.25,1.00,0.0,0.0,0.0,...,77.212786,36.59,0,9,18,2022,261,37,-0.976011,-0.217723


# Additional Engineering

## 1. 카테고리 형 지정

In [17]:
def fit_model(cls, params, tr, vl, ts=None, categorical_features=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], 
        y=tr[TARGET_NAME], 
        sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks,
        categorical_feature=categorical_features  # 카테고리형 변수 지정
    )
    
    vl_preds = np.clip(model.predict(vl[features]), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip(model.predict(ts[features]), 0, np.inf)
    
    if shap_values is None:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
    
    return vl_preds, ts_preds

#### (1) day_of_month, day_of_year, week_no 모두 카테고리 지정

In [16]:
categorical_features = ['day_of_month', 'day_of_year', 'week_no']

In [21]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts, categorical_features)

[20]	valid_0's l1: 36.6122
[40]	valid_0's l1: 27.5759
[60]	valid_0's l1: 24.9621
[80]	valid_0's l1: 23.8264
[100]	valid_0's l1: 23.2459
[120]	valid_0's l1: 22.8791
[140]	valid_0's l1: 22.6359
[160]	valid_0's l1: 22.4829
[180]	valid_0's l1: 22.3625
[200]	valid_0's l1: 22.2408
[220]	valid_0's l1: 22.1679
[240]	valid_0's l1: 22.1039
[260]	valid_0's l1: 22.0138
[280]	valid_0's l1: 21.9509
[300]	valid_0's l1: 21.8808
[320]	valid_0's l1: 21.8403
[340]	valid_0's l1: 21.781
[360]	valid_0's l1: 21.7652
[380]	valid_0's l1: 21.7144
[400]	valid_0's l1: 21.6669
[420]	valid_0's l1: 21.6552
[440]	valid_0's l1: 21.6442
[460]	valid_0's l1: 21.6159
[480]	valid_0's l1: 21.5963
[500]	valid_0's l1: 21.5904
[520]	valid_0's l1: 21.5763
[540]	valid_0's l1: 21.5638
[560]	valid_0's l1: 21.5356
[580]	valid_0's l1: 21.5313
[600]	valid_0's l1: 21.5047
[620]	valid_0's l1: 21.4898
[640]	valid_0's l1: 21.479
[660]	valid_0's l1: 21.4601
[680]	valid_0's l1: 21.446
[700]	valid_0's l1: 21.4401
[720]	valid_0's l1: 21.4312

20-1 선에서 더이상 떨어지지 않아서 다음 단계로



#### (2) `week_no`만 추가로 카테고리 지정 | public score: 20.13687
(model val: [7600]	valid_0's l1: 13.0332)



(카테고리컬 피처는 원래 노트북을 따라감)  

```python
cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year',
]
```

In [58]:
categorical_features = ['week_no']

In [ ]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts, categorical_features)

##### refit and submission 생성

In [ ]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.3394
[40]	valid_0's l1: 26.3549
[60]	valid_0's l1: 23.0208
[80]	valid_0's l1: 21.5567
[100]	valid_0's l1: 20.7145
[120]	valid_0's l1: 20.1688
[140]	valid_0's l1: 19.7721
[160]	valid_0's l1: 19.4936
[180]	valid_0's l1: 19.294
[200]	valid_0's l1: 19.1151
[220]	valid_0's l1: 18.9608
[240]	valid_0's l1: 18.8292
[260]	valid_0's l1: 18.6473
[280]	valid_0's l1: 18.4986
[300]	valid_0's l1: 18.3801
[320]	valid_0's l1: 18.2612
[340]	valid_0's l1: 18.1535
[360]	valid_0's l1: 18.0359
[380]	valid_0's l1: 17.9497
[400]	valid_0's l1: 17.8606
[420]	valid_0's l1: 17.7663
[440]	valid_0's l1: 17.6886
[460]	valid_0's l1: 17.6211
[480]	valid_0's l1: 17.5429
[500]	valid_0's l1: 17.4861
[520]	valid_0's l1: 17.4366
[540]	valid_0's l1: 17.375
[560]	valid_0's l1: 17.3202
[580]	valid_0's l1: 17.2878
[600]	valid_0's l1: 17.2339
[620]	valid_0's l1: 17.1888
[640]	valid_0's l1: 17.1235
[660]	valid_0's l1: 17.0733
[680]	valid_0's l1: 17.0033
[700]	valid_0's l1: 16.9577
[720]	valid_0's l1: 16.908

In [ ]:
subname = '-quacking'
vername = '-engineering-1-2'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,222.348522
1,1_2024-06-04,182.626952
2,1_2024-06-05,178.728645
3,1_2024-06-06,202.901856
4,1_2024-06-07,231.428544
...,...,...
95,11_2024-06-15,28.476046
96,11_2024-06-16,25.514846
97,12_2024-06-03,36.680830
98,12_2024-06-04,39.379919


#### (3) `day_of_week`를 카테고리에서 제외

(model val: [7600]	valid_0's l1: 13.1243)

In [51]:
cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    # 'day_of_week', 
    'month', 'year',
]


all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')


del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

categorical_features = ['week_no']

for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [12]:
def fit_model(cls, params, tr, vl, ts=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], y = tr[TARGET_NAME], sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks
    )
    
    vl_preds = np.clip((model.predict(vl[features])), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip((model.predict(ts[features])), 0, np.inf)
    
    if shap_values is None:
        # Calculate shap values on first model, first fold:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
        pass
    
    return vl_preds, ts_preds

(다음 셀은 잘못누른 실행...)

In [ ]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

In [54]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.3394
[40]	valid_0's l1: 26.385
[60]	valid_0's l1: 23.2408
[80]	valid_0's l1: 21.7454
[100]	valid_0's l1: 20.8846
[120]	valid_0's l1: 20.3094
[140]	valid_0's l1: 19.8691
[160]	valid_0's l1: 19.5826
[180]	valid_0's l1: 19.375
[200]	valid_0's l1: 19.1811
[220]	valid_0's l1: 18.9941
[240]	valid_0's l1: 18.8217
[260]	valid_0's l1: 18.633
[280]	valid_0's l1: 18.5407
[300]	valid_0's l1: 18.3826
[320]	valid_0's l1: 18.2898
[340]	valid_0's l1: 18.206
[360]	valid_0's l1: 18.1295
[380]	valid_0's l1: 17.9966
[400]	valid_0's l1: 17.9174
[420]	valid_0's l1: 17.8379
[440]	valid_0's l1: 17.7724
[460]	valid_0's l1: 17.6918
[480]	valid_0's l1: 17.615
[500]	valid_0's l1: 17.5428
[520]	valid_0's l1: 17.4617
[540]	valid_0's l1: 17.3998
[560]	valid_0's l1: 17.3484
[580]	valid_0's l1: 17.2821
[600]	valid_0's l1: 17.2409
[620]	valid_0's l1: 17.1522
[640]	valid_0's l1: 17.0957
[660]	valid_0's l1: 17.0326
[680]	valid_0's l1: 16.9862
[700]	valid_0's l1: 16.9401
[720]	valid_0's l1: 16.9028
[

In [ ]:
subname = '-quacking'
vername = '-engineering-1-3'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

```
id	sales_hat
0	1_2024-06-03	228.048347
1	1_2024-06-04	176.919593
2	1_2024-06-05	170.139284
3	1_2024-06-06	207.391238
4	1_2024-06-07	229.711799
...	...	...
95	11_2024-06-15	29.251146
96	11_2024-06-16	25.973550
97	12_2024-06-03	36.289774
98	12_2024-06-04	39.387360
99	12_2024-06-05	39.548399
100 rows × 2 columns
```
과 다른가?

### 2. 부분적 데이터 사용

####  (1) 6월 데이터만 학습에 사용 | public score: 23.23711  ❌

(model val: [7600]	valid_0's l1: 7.27783)

In [23]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

all_data.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,...,name_w_weighted,day_sales_m1,day_of_week,month,day_of_month,year,day_of_year,week_no,sin_dayofyear,cos_dayofyear
0,4605,2022-03-01,Prague_3,4683.0,22.40,76.04,1.00,0.0,0.0,0.0,...,111.546148,43.09,2,3,1,2022,60,9,0.858764,0.512371
1,4605,2022-04-20,Prague_3,4691.0,33.88,14.00,0.13,0.0,0.0,0.0,...,115.675143,48.03,3,4,20,2022,110,16,0.948362,-0.317191
2,4605,2023-01-05,Prague_3,4997.0,45.65,90.59,0.47,0.0,0.0,0.0,...,104.884786,49.51,4,1,5,2023,5,1,0.085965,0.996298
3,4605,2022-03-06,Prague_3,4538.0,28.64,80.98,1.00,0.0,0.0,0.0,...,111.546148,35.74,0,3,6,2022,65,9,0.899631,0.436651
4,4605,2022-09-18,Prague_3,4707.0,51.88,79.25,1.00,0.0,0.0,0.0,...,77.212786,36.59,0,9,18,2022,261,37,-0.976011,-0.217723


In [24]:
ts = all_data.query('rowtype == "test"')
# 6월 데이터만 사용
tr = all_data[all_data['month']==6].query('rowtype == "train" and dt_fold < 97')
vl = all_data[all_data['month']==6].query('rowtype == "train" and dt_fold >= 97')

In [25]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 100 metric: 31.7399


In [26]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 35.1425
[40]	valid_0's l1: 26.1868
[60]	valid_0's l1: 23.5813
[80]	valid_0's l1: 22.7037
[100]	valid_0's l1: 22.4191
[120]	valid_0's l1: 22.3239
[140]	valid_0's l1: 22.2972
[160]	valid_0's l1: 22.2226
[180]	valid_0's l1: 22.2508
[200]	valid_0's l1: 22.2494
[220]	valid_0's l1: 22.2601
[240]	valid_0's l1: 22.2452
[260]	valid_0's l1: 22.2252
[280]	valid_0's l1: 22.2199
[300]	valid_0's l1: 22.1993
[320]	valid_0's l1: 22.2048
[340]	valid_0's l1: 22.1894
[360]	valid_0's l1: 22.2366
[380]	valid_0's l1: 22.2402
[400]	valid_0's l1: 22.2212
[420]	valid_0's l1: 22.1652
[440]	valid_0's l1: 22.175
[460]	valid_0's l1: 22.181
[480]	valid_0's l1: 22.1568
[500]	valid_0's l1: 22.1522
[520]	valid_0's l1: 22.1345
[540]	valid_0's l1: 22.153
[560]	valid_0's l1: 22.1513
[580]	valid_0's l1: 22.1606
[600]	valid_0's l1: 22.1523
[620]	valid_0's l1: 22.1331
[640]	valid_0's l1: 22.1127
[660]	valid_0's l1: 22.1055
[680]	valid_0's l1: 22.1067
[700]	valid_0's l1: 22.1181
[720]	valid_0's l1: 22.125


In [27]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 33.9877
[40]	valid_0's l1: 24.0422
[60]	valid_0's l1: 20.9204
[80]	valid_0's l1: 19.646
[100]	valid_0's l1: 19.1425
[120]	valid_0's l1: 18.7428
[140]	valid_0's l1: 18.4264
[160]	valid_0's l1: 18.1802
[180]	valid_0's l1: 17.9048
[200]	valid_0's l1: 17.7161
[220]	valid_0's l1: 17.5306
[240]	valid_0's l1: 17.3053
[260]	valid_0's l1: 17.0571
[280]	valid_0's l1: 16.817
[300]	valid_0's l1: 16.698
[320]	valid_0's l1: 16.5445
[340]	valid_0's l1: 16.408
[360]	valid_0's l1: 16.2907
[380]	valid_0's l1: 16.19
[400]	valid_0's l1: 16.0372
[420]	valid_0's l1: 15.9281
[440]	valid_0's l1: 15.838
[460]	valid_0's l1: 15.7547
[480]	valid_0's l1: 15.6286
[500]	valid_0's l1: 15.5555
[520]	valid_0's l1: 15.4157
[540]	valid_0's l1: 15.3247
[560]	valid_0's l1: 15.1837
[580]	valid_0's l1: 15.1097
[600]	valid_0's l1: 15.0113
[620]	valid_0's l1: 14.9158
[640]	valid_0's l1: 14.7983
[660]	valid_0's l1: 14.7192
[680]	valid_0's l1: 14.6294
[700]	valid_0's l1: 14.5699
[720]	valid_0's l1: 14.5075
[74

In [28]:
subname = '-quacking'
vername = '-engineering-2-1'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,148.095918
1,1_2024-06-04,131.959832
2,1_2024-06-05,150.070635
3,1_2024-06-06,172.680814
4,1_2024-06-07,165.980765
...,...,...
95,11_2024-06-15,28.039326
96,11_2024-06-16,28.302112
97,12_2024-06-03,41.149318
98,12_2024-06-04,44.208904


### 3. 날짜 피처 sin cos 정보 추가

*이 부분부터 재시작*

In [1]:
import sys
import os
import gc

import warnings
warnings.filterwarnings('ignore')

import random
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import duckdb

import lightgbm as lgb
import xgboost as xgb
import catboost as cat

import shap

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
from plotnine import * 
from mizani.breaks import date_breaks
from mizani.formatters import date_format

import duckdb
duckdb.query('PRAGMA disable_progress_bar;')


RANDOM_STATE = 1966
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(RANDOM_STATE)

class Shhh:
    # some of these models are still quite chatty even after disabling logging
    # we use this to swallow the printed output.
    # see: https://stackoverflow.com/questions/72346178/how-to-suppress-automatically-generated-output-from-a-python-code
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr
        
%matplotlib inline
%config InlineBackend.figure_format='retina'

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

BASEPATH = '/Users/djroz/Desktop/kaggle_study/rohlik-sales-v2-data/'

# borrowed from https://www.kaggle.com/code/samvelkoch/rohlik2-lama-v6-weighted

calendar = duckdb.query(f'''
    SELECT 
       t.* exclude(holiday_name),
       coalesce(t.holiday_name, '-') as holiday_name
    FROM 
        read_csv('{BASEPATH + 'calendar.csv'}') t
    ''').df()

from datetime import datetime
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    df['school'] = df['winter_school_holidays'] + df['school_holidays']
    return df
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = (
    pd.concat(processed_dfs)
    .sort_values('date')
    .drop(columns=['winter_school_holidays', 'school_holidays'])
    #.query('date <= "2024-06-02"')
    .reset_index(drop=True))

In [2]:
# read and join data

duckdb.query(f'''
    create or replace table sales as
    SELECT 
        s.*,
        'train' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_train.csv'}') s
    WHERE
        -- 52 rows have a null sales observation. 
        -- we just drop them.
        s.sales is not null
    UNION ALL BY NAME
    SELECT 
        s.*,
        'test' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_test.csv'}') s
    ''')


duckdb.query(f'''
    create or replace table inventory as
    SELECT 
       t.* exclude(L1_category_name_en, L2_category_name_en, L3_category_name_en, L4_category_name_en),
       L1_category_name_en as l1_cat,
       L1_category_name_en || '-' ||  L2_category_name_en as l2_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en as l3_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en || '-' ||  L3_category_name_en as l4_cat
    FROM 
        read_csv('{BASEPATH + 'inventory.csv'}') t
    ''')

duckdb.query(f'''
    create or replace table calendar as
    SELECT 
       *
    FROM 
        calendar_extended
    ''')

duckdb.query(f'''
    create or replace table test_weights as
    SELECT 
       *
    FROM 
        read_csv('{BASEPATH + 'test_weights.csv'}') t
    ''')

duckdb.query(f'''
    create or replace view all_data as
    SELECT 
        s.*,
        c.* exclude(warehouse, date),
        i.* exclude(warehouse, unique_id),
        w.* exclude(unique_id),
        -- group into fortnights
        -- The whole of test is a single fold
        date_diff('day', '2020-07-20', s.date) // 14 as dt_fold,
        date_diff('day', '2020-07-20', s.date) % 14  as dt_fold_day,
    FROM 
        sales s
        join calendar c on c.warehouse = s.warehouse and c.date = s.date
        join inventory i on i.warehouse = s.warehouse and i.unique_id = s.unique_id
        join test_weights w on w.unique_id = s.unique_id
''')

In [3]:
# add groupings for previous weeks:

groupings = ['l1_cat', 'l2_cat', 'l3_cat', 'l4_cat', 'name']
for grouping in groupings:
    # without warehouse
    duckdb.query(f'''
        create or replace table {grouping}_hist as
        select dt_fold, {grouping}, mean(sales) as sales
        from all_data
        where dt_fold > 0
        group by dt_fold, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')
    # with warehouse
    duckdb.query(f'''
        create or replace table {grouping}_whist as
        select dt_fold, warehouse, {grouping}, mean(sales) as sales, stddev(sales) as sales_sd,
        from all_data
        where dt_fold > 0
        group by dt_fold, warehouse, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')

# same day:
duckdb.query(f'''
    create or replace table name_whist_day as
    select dt_fold, dt_fold_day, warehouse, name, sum(sales) as sales
    from all_data
    where dt_fold > 0
    group by dt_fold, dt_fold_day, warehouse, name
    having sum(sales) is not null
    order by dt_fold, warehouse, name''')

In [4]:
duckdb.query(f'''
    create or replace table fe_data as
    select 
        a.*,
        -- sales of similar products in previous fortnight.
        -- overall
        coalesce(l1_cat_hist.sales, 0) as l1_cat_m1,
        coalesce(l2_cat_hist.sales, 0) as l2_cat_m1,
        coalesce(l3_cat_hist.sales, 0) as l3_cat_m1,
        coalesce(l4_cat_hist.sales, 0) as l4_cat_m1,
        coalesce(name_hist.sales, 0) as name_m1,
        -- by warehouse
        coalesce(l1_cat_whist.sales, 0) as l1_cat_wm1,
        coalesce(l2_cat_whist.sales, 0) as l2_cat_wm1,
        coalesce(l3_cat_whist.sales, 0) as l3_cat_wm1,
        coalesce(l4_cat_whist.sales, 0) as l4_cat_wm1,
        -- name & warehouse last 3
        coalesce(m1.sales, 0) as name_wm1,
        coalesce(m1.sales_sd, 0) as name_sd_wm1,
        coalesce(m2.sales, 0) as name_wm2,
        coalesce(m3.sales, 0) as name_wm3,
        coalesce(m4.sales, 0) as name_wm4,
        coalesce(m5.sales, 0) as name_wm5,
        (coalesce(m1.sales, 0) * 1
         + coalesce(m2.sales, 0) * 0.8
         + coalesce(m3.sales, 0) * 0.6
         + coalesce(m4.sales, 0) * 0.3
         + coalesce(m5.sales, 0) * 0.1 ) as name_w_weighted,
        coalesce(name_whist_day.sales, 0) as day_sales_m1,
        extract(dayofweek FROM date) AS day_of_week, 
        extract(month FROM date) AS month, 
        extract(day FROM date) AS day_of_month,
        extract(year FROM date) as year,
        extract(dayofyear FROM date) as day_of_year,
        extract(week FROM date) as week_no,
    from 
        all_data a
        -- overall all warehouses:
        left join l1_cat_hist 
            on l1_cat_hist.dt_fold + 1 = a.dt_fold 
                and l1_cat_hist.l1_cat = a.l1_cat
        left join l2_cat_hist 
            on l2_cat_hist.dt_fold + 1 = a.dt_fold 
                and l2_cat_hist.l2_cat = a.l2_cat
        left join l3_cat_hist 
            on l3_cat_hist.dt_fold + 1 = a.dt_fold 
                and l3_cat_hist.l3_cat = a.l3_cat
        left join l4_cat_hist 
            on l4_cat_hist.dt_fold + 1 = a.dt_fold 
                and l4_cat_hist.l4_cat = a.l4_cat
        left join name_hist 
            on name_hist.dt_fold + 1 = a.dt_fold 
                and name_hist.name = a.name
        -- by warehouse
        left join l1_cat_whist 
            on l1_cat_whist.dt_fold + 1 = a.dt_fold 
                and l1_cat_whist.warehouse = a.warehouse 
                and l1_cat_whist.l1_cat = a.l1_cat
        left join l2_cat_whist
            on l2_cat_whist.dt_fold + 1 = a.dt_fold 
                and l2_cat_whist.warehouse = a.warehouse 
                and l2_cat_whist.l2_cat = a.l2_cat
        left join l3_cat_whist 
            on l3_cat_whist.dt_fold + 1 = a.dt_fold 
                and l3_cat_whist.warehouse = a.warehouse 
                and l3_cat_whist.l3_cat = a.l3_cat
        left join l4_cat_whist 
            on l4_cat_whist.dt_fold + 1 = a.dt_fold 
                and l4_cat_whist.warehouse = a.warehouse 
                and l4_cat_whist.l4_cat = a.l4_cat
        left join name_whist as m1 
            on m1.dt_fold + 1 = a.dt_fold 
                and m1.warehouse = a.warehouse 
                and m1.name = a.name
        left join name_whist as m2 
            on m2.dt_fold + 2 = a.dt_fold 
                and m2.warehouse = a.warehouse 
                and m2.name = a.name
        left join name_whist as m3 
            on m3.dt_fold + 3 = a.dt_fold 
                and m3.warehouse = a.warehouse 
                and m3.name = a.name
        left join name_whist as m4 
            on m4.dt_fold + 4 = a.dt_fold 
                and m4.warehouse = a.warehouse 
                and m4.name = a.name
        left join name_whist as m5 
            on m5.dt_fold + 5 = a.dt_fold 
                and m5.warehouse = a.warehouse 
                and m5.name = a.name
        left join name_whist_day 
            on name_whist_day.dt_fold + 1 = a.dt_fold
                and name_whist_day.dt_fold_day = a.dt_fold_day
                and name_whist_day.warehouse = a.warehouse 
                and name_whist_day.name = a.name
    where
        a.dt_fold > 2
''')

In [5]:
TARGET_NAME = 'sales'

cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year',
]

features = [
    'warehouse', 'total_orders',
    'sell_price_main', 'type_0_discount', 'type_1_discount',
    'type_2_discount', 'type_3_discount', 'type_4_discount',
    'type_5_discount', 'type_6_discount',  
    'holiday',
    'shops_closed', 
    'days_to_holiday',
    'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    #'l1_cat_m1', 'l2_cat_m1', 'l3_cat_m1', 
    'l4_cat_m1', 'name_m1', 
    #'l1_cat_wm1', 'l2_cat_wm1', 'l3_cat_wm1', 
    'l4_cat_wm1', 
    'name_wm1', 'name_sd_wm1',
    'name_wm2', 'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted',
    'day_sales_m1',
    'day_of_week', 'month', 'day_of_month', 'year',
    'day_of_year', 'sin_dayofyear', 'cos_dayofyear',
    'week_no',        
]

#### (1) month 정보 sin cos 정보 추가

(model val: [7600]	valid_0's l1: 13.142)

In [6]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

In [7]:
### 월의 길이에 따라 sin cos 계산
from calendar import monthrange
# 각 행별로 해당 월의 최대 일 수를 구하여 n으로 사용
all_data['days_in_month'] = all_data.apply(lambda row: monthrange(row['year'], row['month'])[1], axis=1)
# 월 주기성 피처 추가
all_data['sin_dayofmonth'] = np.sin(2 * np.pi * all_data['day_of_month'] / all_data['days_in_month'])
all_data['cos_dayofmonth'] = np.cos(2 * np.pi * all_data['day_of_month'] / all_data['days_in_month'])


ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')

# garbage collection
del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 60), (3759180, 60), (190794, 60))

In [17]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3759180 entries, 0 to 3996994
Data columns (total 60 columns):
 #   Column                Dtype         
---  ------                -----         
 0   unique_id             int64         
 1   date                  datetime64[us]
 2   warehouse             category      
 3   total_orders          float64       
 4   sales                 float64       
 5   sell_price_main       float64       
 6   availability          float64       
 7   type_0_discount       float64       
 8   type_1_discount       float64       
 9   type_2_discount       float64       
 10  type_3_discount       float64       
 11  type_4_discount       float64       
 12  type_5_discount       float64       
 13  type_6_discount       float64       
 14  rowtype               object        
 15  holiday               int64         
 16  shops_closed          int64         
 17  holiday_name          object        
 18  days_to_holiday       float64       
 19  days_

In [18]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [19]:
def fit_model(cls, params, tr, vl, ts=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], y = tr[TARGET_NAME], sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks
    )
    
    vl_preds = np.clip((model.predict(vl[features])), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip((model.predict(ts[features])), 0, np.inf)
    
    if shap_values is None:
        # Calculate shap values on first model, first fold:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
        pass
    
    return vl_preds, ts_preds

In [20]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 36.6844
[40]	valid_0's l1: 27.1288
[60]	valid_0's l1: 24.2441
[80]	valid_0's l1: 22.9079
[100]	valid_0's l1: 22.2347
[120]	valid_0's l1: 21.7802
[140]	valid_0's l1: 21.4712
[160]	valid_0's l1: 21.2711
[180]	valid_0's l1: 21.142
[200]	valid_0's l1: 21.0214
[220]	valid_0's l1: 20.9406
[240]	valid_0's l1: 20.8075
[260]	valid_0's l1: 20.7469
[280]	valid_0's l1: 20.6342
[300]	valid_0's l1: 20.5937
[320]	valid_0's l1: 20.544
[340]	valid_0's l1: 20.4712
[360]	valid_0's l1: 20.4208
[380]	valid_0's l1: 20.3899
[400]	valid_0's l1: 20.3527
[420]	valid_0's l1: 20.3423
[440]	valid_0's l1: 20.3182
[460]	valid_0's l1: 20.3085
[480]	valid_0's l1: 20.2712
[500]	valid_0's l1: 20.2457
[520]	valid_0's l1: 20.2305
[540]	valid_0's l1: 20.2122
[560]	valid_0's l1: 20.1881
[580]	valid_0's l1: 20.1745
[600]	valid_0's l1: 20.157
[620]	valid_0's l1: 20.1318
[640]	valid_0's l1: 20.1296
[660]	valid_0's l1: 20.1246
[680]	valid_0's l1: 20.1125
[700]	valid_0's l1: 20.0951
[720]	valid_0's l1: 20.0931

In [21]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.409
[40]	valid_0's l1: 26.4032
[60]	valid_0's l1: 23.2334
[80]	valid_0's l1: 21.7402
[100]	valid_0's l1: 20.9172
[120]	valid_0's l1: 20.3688
[140]	valid_0's l1: 19.9693
[160]	valid_0's l1: 19.6924
[180]	valid_0's l1: 19.4717
[200]	valid_0's l1: 19.2735
[220]	valid_0's l1: 19.1204
[240]	valid_0's l1: 18.9386
[260]	valid_0's l1: 18.7905
[280]	valid_0's l1: 18.6117
[300]	valid_0's l1: 18.454
[320]	valid_0's l1: 18.3397
[340]	valid_0's l1: 18.204
[360]	valid_0's l1: 18.1053
[380]	valid_0's l1: 18.0192
[400]	valid_0's l1: 17.9512
[420]	valid_0's l1: 17.8891
[440]	valid_0's l1: 17.8363
[460]	valid_0's l1: 17.7625
[480]	valid_0's l1: 17.7029
[500]	valid_0's l1: 17.6601
[520]	valid_0's l1: 17.6014
[540]	valid_0's l1: 17.5211
[560]	valid_0's l1: 17.4648
[580]	valid_0's l1: 17.3978
[600]	valid_0's l1: 17.359
[620]	valid_0's l1: 17.2887
[640]	valid_0's l1: 17.2235
[660]	valid_0's l1: 17.1745
[680]	valid_0's l1: 17.1267
[700]	valid_0's l1: 17.0864
[720]	valid_0's l1: 17.0403


In [22]:
subname = '-quacking'
vername = '-engineering-3-1'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,225.363985
1,1_2024-06-04,186.360031
2,1_2024-06-05,182.488646
3,1_2024-06-06,205.433718
4,1_2024-06-07,244.926748
5,1_2024-06-08,174.748061
6,1_2024-06-09,172.964844
7,1_2024-06-10,216.910727
8,1_2024-06-11,178.127022
9,1_2024-06-12,191.696418


#### (2) `day_of_month` sin/cos + week_no 카테고리화 | public score: 20.33805

(model val: [7600]	valid_0's l1: 12.9261)

In [10]:
TARGET_NAME = 'sales'

cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year',
    'week_no' # 추가
]

features = [
    'warehouse', 'total_orders',
    'sell_price_main', 'type_0_discount', 'type_1_discount',
    'type_2_discount', 'type_3_discount', 'type_4_discount',
    'type_5_discount', 'type_6_discount',  
    'holiday',
    'shops_closed', 
    'days_to_holiday',
    'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    #'l1_cat_m1', 'l2_cat_m1', 'l3_cat_m1', 
    'l4_cat_m1', 'name_m1', 
    #'l1_cat_wm1', 'l2_cat_wm1', 'l3_cat_wm1', 
    'l4_cat_wm1', 
    'name_wm1', 'name_sd_wm1',
    'name_wm2', 'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted',
    'day_sales_m1',
    'day_of_week', 'month', 'day_of_month', 'year',
    'day_of_year', 'sin_dayofyear', 'cos_dayofyear',
    'week_no',        
]

In [ ]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

In [ ]:
### 월의 길이에 따라 sin cos 계산
from calendar import monthrange
# 각 행별로 해당 월의 최대 일 수를 구하여 n으로 사용
all_data['days_in_month'] = all_data.apply(lambda row: monthrange(row['year'], row['month'])[1], axis=1)
# 월 주기성 피처 추가
all_data['sin_dayofmonth'] = np.sin(2 * np.pi * all_data['day_of_month'] / all_data['days_in_month'])
all_data['cos_dayofmonth'] = np.cos(2 * np.pi * all_data['day_of_month'] / all_data['days_in_month'])


ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')

# garbage collection
del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 60), (3759180, 60), (190794, 60))

In [ ]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [ ]:
shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 36.685
[40]	valid_0's l1: 27.2197
[60]	valid_0's l1: 24.3092
[80]	valid_0's l1: 22.9197
[100]	valid_0's l1: 22.2061
[120]	valid_0's l1: 21.7817
[140]	valid_0's l1: 21.5079
[160]	valid_0's l1: 21.331
[180]	valid_0's l1: 21.1767
[200]	valid_0's l1: 21.0497
[220]	valid_0's l1: 20.9841
[240]	valid_0's l1: 20.9048
[260]	valid_0's l1: 20.8259
[280]	valid_0's l1: 20.7756
[300]	valid_0's l1: 20.713
[320]	valid_0's l1: 20.6629
[340]	valid_0's l1: 20.6323
[360]	valid_0's l1: 20.6187
[380]	valid_0's l1: 20.5964
[400]	valid_0's l1: 20.5634
[420]	valid_0's l1: 20.5473
[440]	valid_0's l1: 20.5131
[460]	valid_0's l1: 20.4875
[480]	valid_0's l1: 20.4456
[500]	valid_0's l1: 20.4225
[520]	valid_0's l1: 20.4049
[540]	valid_0's l1: 20.3864
[560]	valid_0's l1: 20.3844
[580]	valid_0's l1: 20.3625
[600]	valid_0's l1: 20.3513
[620]	valid_0's l1: 20.3474
[640]	valid_0's l1: 20.3449
[660]	valid_0's l1: 20.3404
[680]	valid_0's l1: 20.3353
[700]	valid_0's l1: 20.3176
[720]	valid_0's l1: 20.3057

In [ ]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.4249
[40]	valid_0's l1: 26.4356
[60]	valid_0's l1: 23.1784
[80]	valid_0's l1: 21.6
[100]	valid_0's l1: 20.743
[120]	valid_0's l1: 20.1971
[140]	valid_0's l1: 19.841
[160]	valid_0's l1: 19.5478
[180]	valid_0's l1: 19.3642
[200]	valid_0's l1: 19.1679
[220]	valid_0's l1: 19.028
[240]	valid_0's l1: 18.8902
[260]	valid_0's l1: 18.7396
[280]	valid_0's l1: 18.5563
[300]	valid_0's l1: 18.441
[320]	valid_0's l1: 18.3117
[340]	valid_0's l1: 18.1995
[360]	valid_0's l1: 18.1029
[380]	valid_0's l1: 17.9964
[400]	valid_0's l1: 17.8888
[420]	valid_0's l1: 17.8304
[440]	valid_0's l1: 17.7382
[460]	valid_0's l1: 17.6634
[480]	valid_0's l1: 17.5881
[500]	valid_0's l1: 17.5344
[520]	valid_0's l1: 17.4715
[540]	valid_0's l1: 17.4053
[560]	valid_0's l1: 17.3455
[580]	valid_0's l1: 17.2689
[600]	valid_0's l1: 17.1979
[620]	valid_0's l1: 17.1602
[640]	valid_0's l1: 17.0994
[660]	valid_0's l1: 17.0365
[680]	valid_0's l1: 16.9922
[700]	valid_0's l1: 16.9553
[720]	valid_0's l1: 16.8983
[74

In [ ]:
subname = '-quacking'
vername = '-engineering-3-2'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,207.882296
1,1_2024-06-04,175.641835
2,1_2024-06-05,162.968967
3,1_2024-06-06,179.166866
4,1_2024-06-07,213.045305
5,1_2024-06-08,154.787678
6,1_2024-06-09,159.816463
7,1_2024-06-10,211.039606
8,1_2024-06-11,174.594270
9,1_2024-06-12,184.584890


### 4. lagged sales 피처 추가

*다시 재시작*

In [1]:
import sys
import os
import gc

import warnings
warnings.filterwarnings('ignore')

import random
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import duckdb

import lightgbm as lgb
import xgboost as xgb
import catboost as cat

import shap

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
from plotnine import * 
from mizani.breaks import date_breaks
from mizani.formatters import date_format

import duckdb
duckdb.query('PRAGMA disable_progress_bar;')


RANDOM_STATE = 1966
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(RANDOM_STATE)

class Shhh:
    # some of these models are still quite chatty even after disabling logging
    # we use this to swallow the printed output.
    # see: https://stackoverflow.com/questions/72346178/how-to-suppress-automatically-generated-output-from-a-python-code
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr
        
%matplotlib inline
%config InlineBackend.figure_format='retina'

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

BASEPATH = '/Users/djroz/Desktop/kaggle_study/rohlik-sales-v2-data/'

# borrowed from https://www.kaggle.com/code/samvelkoch/rohlik2-lama-v6-weighted

calendar = duckdb.query(f'''
    SELECT 
       t.* exclude(holiday_name),
       coalesce(t.holiday_name, '-') as holiday_name
    FROM 
        read_csv('{BASEPATH + 'calendar.csv'}') t
    ''').df()

from datetime import datetime
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    df['school'] = df['winter_school_holidays'] + df['school_holidays']
    return df
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = (
    pd.concat(processed_dfs)
    .sort_values('date')
    .drop(columns=['winter_school_holidays', 'school_holidays'])
    #.query('date <= "2024-06-02"')
    .reset_index(drop=True))

In [2]:
# read and join data

duckdb.query(f'''
    create or replace table sales as
    SELECT 
        s.*,
        'train' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_train.csv'}') s
    WHERE
        -- 52 rows have a null sales observation. 
        -- we just drop them.
        s.sales is not null
    UNION ALL BY NAME
    SELECT 
        s.*,
        'test' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_test.csv'}') s
    ''')


duckdb.query(f'''
    create or replace table inventory as
    SELECT 
       t.* exclude(L1_category_name_en, L2_category_name_en, L3_category_name_en, L4_category_name_en),
       L1_category_name_en as l1_cat,
       L1_category_name_en || '-' ||  L2_category_name_en as l2_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en as l3_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en || '-' ||  L3_category_name_en as l4_cat
    FROM 
        read_csv('{BASEPATH + 'inventory.csv'}') t
    ''')

duckdb.query(f'''
    create or replace table calendar as
    SELECT 
       *
    FROM 
        calendar_extended
    ''')

duckdb.query(f'''
    create or replace table test_weights as
    SELECT 
       *
    FROM 
        read_csv('{BASEPATH + 'test_weights.csv'}') t
    ''')

duckdb.query(f'''
    create or replace view all_data as
    SELECT 
        s.*,
        c.* exclude(warehouse, date),
        i.* exclude(warehouse, unique_id),
        w.* exclude(unique_id),
        -- group into fortnights
        -- The whole of test is a single fold
        date_diff('day', '2020-07-20', s.date) // 14 as dt_fold,
        date_diff('day', '2020-07-20', s.date) % 14  as dt_fold_day,
    FROM 
        sales s
        join calendar c on c.warehouse = s.warehouse and c.date = s.date
        join inventory i on i.warehouse = s.warehouse and i.unique_id = s.unique_id
        join test_weights w on w.unique_id = s.unique_id
''')

In [3]:
# add groupings for previous weeks:

groupings = ['l1_cat', 'l2_cat', 'l3_cat', 'l4_cat', 'name']
for grouping in groupings:
    # without warehouse
    duckdb.query(f'''
        create or replace table {grouping}_hist as
        select dt_fold, {grouping}, mean(sales) as sales
        from all_data
        where dt_fold > 0
        group by dt_fold, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')
    # with warehouse
    duckdb.query(f'''
        create or replace table {grouping}_whist as
        select dt_fold, warehouse, {grouping}, mean(sales) as sales, stddev(sales) as sales_sd,
        from all_data
        where dt_fold > 0
        group by dt_fold, warehouse, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')

# same day:
duckdb.query(f'''
    create or replace table name_whist_day as
    select dt_fold, dt_fold_day, warehouse, name, sum(sales) as sales
    from all_data
    where dt_fold > 0
    group by dt_fold, dt_fold_day, warehouse, name
    having sum(sales) is not null
    order by dt_fold, warehouse, name''')

In [4]:
duckdb.query(f'''
    create or replace table fe_data as
    select 
        a.*,
        -- sales of similar products in previous fortnight.
        -- overall
        coalesce(l1_cat_hist.sales, 0) as l1_cat_m1,
        coalesce(l2_cat_hist.sales, 0) as l2_cat_m1,
        coalesce(l3_cat_hist.sales, 0) as l3_cat_m1,
        coalesce(l4_cat_hist.sales, 0) as l4_cat_m1,
        coalesce(name_hist.sales, 0) as name_m1,
        -- by warehouse
        coalesce(l1_cat_whist.sales, 0) as l1_cat_wm1,
        coalesce(l2_cat_whist.sales, 0) as l2_cat_wm1,
        coalesce(l3_cat_whist.sales, 0) as l3_cat_wm1,
        coalesce(l4_cat_whist.sales, 0) as l4_cat_wm1,
        -- name & warehouse last 3
        coalesce(m1.sales, 0) as name_wm1,
        coalesce(m1.sales_sd, 0) as name_sd_wm1,
        coalesce(m2.sales, 0) as name_wm2,
        coalesce(m3.sales, 0) as name_wm3,
        coalesce(m4.sales, 0) as name_wm4,
        coalesce(m5.sales, 0) as name_wm5,
        (coalesce(m1.sales, 0) * 1
         + coalesce(m2.sales, 0) * 0.8
         + coalesce(m3.sales, 0) * 0.6
         + coalesce(m4.sales, 0) * 0.3
         + coalesce(m5.sales, 0) * 0.1 ) as name_w_weighted,
        coalesce(name_whist_day.sales, 0) as day_sales_m1,
        extract(dayofweek FROM date) AS day_of_week, 
        extract(month FROM date) AS month, 
        extract(day FROM date) AS day_of_month,
        extract(year FROM date) as year,
        extract(dayofyear FROM date) as day_of_year,
        extract(week FROM date) as week_no,
    from 
        all_data a
        -- overall all warehouses:
        left join l1_cat_hist 
            on l1_cat_hist.dt_fold + 1 = a.dt_fold 
                and l1_cat_hist.l1_cat = a.l1_cat
        left join l2_cat_hist 
            on l2_cat_hist.dt_fold + 1 = a.dt_fold 
                and l2_cat_hist.l2_cat = a.l2_cat
        left join l3_cat_hist 
            on l3_cat_hist.dt_fold + 1 = a.dt_fold 
                and l3_cat_hist.l3_cat = a.l3_cat
        left join l4_cat_hist 
            on l4_cat_hist.dt_fold + 1 = a.dt_fold 
                and l4_cat_hist.l4_cat = a.l4_cat
        left join name_hist 
            on name_hist.dt_fold + 1 = a.dt_fold 
                and name_hist.name = a.name
        -- by warehouse
        left join l1_cat_whist 
            on l1_cat_whist.dt_fold + 1 = a.dt_fold 
                and l1_cat_whist.warehouse = a.warehouse 
                and l1_cat_whist.l1_cat = a.l1_cat
        left join l2_cat_whist
            on l2_cat_whist.dt_fold + 1 = a.dt_fold 
                and l2_cat_whist.warehouse = a.warehouse 
                and l2_cat_whist.l2_cat = a.l2_cat
        left join l3_cat_whist 
            on l3_cat_whist.dt_fold + 1 = a.dt_fold 
                and l3_cat_whist.warehouse = a.warehouse 
                and l3_cat_whist.l3_cat = a.l3_cat
        left join l4_cat_whist 
            on l4_cat_whist.dt_fold + 1 = a.dt_fold 
                and l4_cat_whist.warehouse = a.warehouse 
                and l4_cat_whist.l4_cat = a.l4_cat
        left join name_whist as m1 
            on m1.dt_fold + 1 = a.dt_fold 
                and m1.warehouse = a.warehouse 
                and m1.name = a.name
        left join name_whist as m2 
            on m2.dt_fold + 2 = a.dt_fold 
                and m2.warehouse = a.warehouse 
                and m2.name = a.name
        left join name_whist as m3 
            on m3.dt_fold + 3 = a.dt_fold 
                and m3.warehouse = a.warehouse 
                and m3.name = a.name
        left join name_whist as m4 
            on m4.dt_fold + 4 = a.dt_fold 
                and m4.warehouse = a.warehouse 
                and m4.name = a.name
        left join name_whist as m5 
            on m5.dt_fold + 5 = a.dt_fold 
                and m5.warehouse = a.warehouse 
                and m5.name = a.name
        left join name_whist_day 
            on name_whist_day.dt_fold + 1 = a.dt_fold
                and name_whist_day.dt_fold_day = a.dt_fold_day
                and name_whist_day.warehouse = a.warehouse 
                and name_whist_day.name = a.name
    where
        a.dt_fold > 2
''')

In [5]:
TARGET_NAME = 'sales'

cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year',
]

features = [
    'warehouse', 'total_orders',
    'sell_price_main', 'type_0_discount', 'type_1_discount',
    'type_2_discount', 'type_3_discount', 'type_4_discount',
    'type_5_discount', 'type_6_discount',  
    'holiday',
    'shops_closed', 
    'days_to_holiday',
    'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    #'l1_cat_m1', 'l2_cat_m1', 'l3_cat_m1', 
    'l4_cat_m1', 'name_m1', 
    #'l1_cat_wm1', 'l2_cat_wm1', 'l3_cat_wm1', 
    'l4_cat_wm1', 
    'name_wm1', 'name_sd_wm1',
    'name_wm2', 'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted',
    'day_sales_m1',
    'day_of_week', 'month', 'day_of_month', 'year',
    'day_of_year', 'sin_dayofyear', 'cos_dayofyear',
    'week_no',        
]

#### (1) periods: 2주, 3주, 4주, 8주, 16주, 32주, 1년 단위로 생성

(model val : [7600]	valid_0's l1: 13.0515)

In [17]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

periods = [14, 21, 28, 56, 112, 224, 365]
def add_lag_sales(df_sales):
    for period in periods:
        df_sales[f"sales_lag_{period}"] = df_sales.groupby("unique_id")["sales"].shift(period)
    
    return df_sales

In [18]:
all_data = add_lag_sales(all_data)

ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')

del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 64), (3759180, 64), (190794, 64))

In [ ]:
# tr.info()

In [20]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [23]:
def fit_model(cls, params, tr, vl, ts=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], y = tr[TARGET_NAME], sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks
    )
    
    vl_preds = np.clip((model.predict(vl[features])), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip((model.predict(ts[features])), 0, np.inf)
    
    if shap_values is None:
        # Calculate shap values on first model, first fold:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
        pass
    
    return vl_preds, ts_preds

shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 36.6607
[40]	valid_0's l1: 27.1628
[60]	valid_0's l1: 24.2289
[80]	valid_0's l1: 22.8702
[100]	valid_0's l1: 22.2335
[120]	valid_0's l1: 21.8175
[140]	valid_0's l1: 21.5185
[160]	valid_0's l1: 21.2921
[180]	valid_0's l1: 21.1532
[200]	valid_0's l1: 21.028
[220]	valid_0's l1: 20.9261
[240]	valid_0's l1: 20.8377
[260]	valid_0's l1: 20.7639
[280]	valid_0's l1: 20.6819
[300]	valid_0's l1: 20.6169
[320]	valid_0's l1: 20.5503
[340]	valid_0's l1: 20.5057
[360]	valid_0's l1: 20.4589
[380]	valid_0's l1: 20.4368
[400]	valid_0's l1: 20.3948
[420]	valid_0's l1: 20.3707
[440]	valid_0's l1: 20.3436
[460]	valid_0's l1: 20.3267
[480]	valid_0's l1: 20.3065
[500]	valid_0's l1: 20.2812
[520]	valid_0's l1: 20.2599
[540]	valid_0's l1: 20.2495
[560]	valid_0's l1: 20.237
[580]	valid_0's l1: 20.2259
[600]	valid_0's l1: 20.2068
[620]	valid_0's l1: 20.1957
[640]	valid_0's l1: 20.1784
[660]	valid_0's l1: 20.1572
[680]	valid_0's l1: 20.1401
[700]	valid_0's l1: 20.1275
[720]	valid_0's l1: 20.115

In [24]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.3567
[40]	valid_0's l1: 26.3114
[60]	valid_0's l1: 23.1212
[80]	valid_0's l1: 21.6283
[100]	valid_0's l1: 20.7617
[120]	valid_0's l1: 20.2475
[140]	valid_0's l1: 19.8573
[160]	valid_0's l1: 19.5582
[180]	valid_0's l1: 19.3564
[200]	valid_0's l1: 19.1905
[220]	valid_0's l1: 19.0416
[240]	valid_0's l1: 18.9056
[260]	valid_0's l1: 18.7102
[280]	valid_0's l1: 18.5667
[300]	valid_0's l1: 18.4352
[320]	valid_0's l1: 18.3138
[340]	valid_0's l1: 18.1953
[360]	valid_0's l1: 18.0867
[380]	valid_0's l1: 17.9985
[400]	valid_0's l1: 17.9202
[420]	valid_0's l1: 17.8263
[440]	valid_0's l1: 17.7728
[460]	valid_0's l1: 17.6995
[480]	valid_0's l1: 17.6347
[500]	valid_0's l1: 17.5709
[520]	valid_0's l1: 17.4936
[540]	valid_0's l1: 17.4503
[560]	valid_0's l1: 17.3904
[580]	valid_0's l1: 17.3291
[600]	valid_0's l1: 17.2709
[620]	valid_0's l1: 17.2122
[640]	valid_0's l1: 17.1358
[660]	valid_0's l1: 17.0972
[680]	valid_0's l1: 17.0571
[700]	valid_0's l1: 17.0032
[720]	valid_0's l1: 16.9

In [25]:
subname = '-quacking'
vername = '-engineering-4-1'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,225.235201
1,1_2024-06-04,187.675513
2,1_2024-06-05,183.520426
3,1_2024-06-06,186.831553
4,1_2024-06-07,231.980786
5,1_2024-06-08,175.989611
6,1_2024-06-09,169.740139
7,1_2024-06-10,213.182814
8,1_2024-06-11,186.205543
9,1_2024-06-12,196.320121


#### (2) periods 주기에 `일주일` 전 lagged_sales 추가

(재시작 후 이어서진행)

In [6]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

periods = [7, 14, 21, 28, 56, 112, 224, 365]
def add_lag_sales(df_sales):
    for period in periods:
        df_sales[f"sales_lag_{period}"] = df_sales.groupby("unique_id")["sales"].shift(period)
    
    return df_sales

In [7]:
all_data = add_lag_sales(all_data)

ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')

del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 65), (3759180, 65), (190794, 65))

In [ ]:
# tr.info()

In [8]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [9]:
def fit_model(cls, params, tr, vl, ts=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], y = tr[TARGET_NAME], sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks
    )
    
    vl_preds = np.clip((model.predict(vl[features])), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip((model.predict(ts[features])), 0, np.inf)
    
    if shap_values is None:
        # Calculate shap values on first model, first fold:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
        pass
    
    return vl_preds, ts_preds

shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 36.6591
[40]	valid_0's l1: 27.0966
[60]	valid_0's l1: 24.1998
[80]	valid_0's l1: 22.9145
[100]	valid_0's l1: 22.252
[120]	valid_0's l1: 21.7828
[140]	valid_0's l1: 21.4842
[160]	valid_0's l1: 21.2922
[180]	valid_0's l1: 21.1297
[200]	valid_0's l1: 20.9695
[220]	valid_0's l1: 20.8811
[240]	valid_0's l1: 20.7795
[260]	valid_0's l1: 20.6881
[280]	valid_0's l1: 20.6194
[300]	valid_0's l1: 20.561
[320]	valid_0's l1: 20.5212
[340]	valid_0's l1: 20.4604
[360]	valid_0's l1: 20.4548
[380]	valid_0's l1: 20.4236
[400]	valid_0's l1: 20.3887
[420]	valid_0's l1: 20.3463
[440]	valid_0's l1: 20.316
[460]	valid_0's l1: 20.3086
[480]	valid_0's l1: 20.2991
[500]	valid_0's l1: 20.2762
[520]	valid_0's l1: 20.2526
[540]	valid_0's l1: 20.2455
[560]	valid_0's l1: 20.243
[580]	valid_0's l1: 20.2225
[600]	valid_0's l1: 20.2156
[620]	valid_0's l1: 20.1913
[640]	valid_0's l1: 20.1739
[660]	valid_0's l1: 20.1592
[680]	valid_0's l1: 20.1461
[700]	valid_0's l1: 20.1385
[720]	valid_0's l1: 20.1213


In [10]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.3414
[40]	valid_0's l1: 26.341
[60]	valid_0's l1: 23.1233
[80]	valid_0's l1: 21.5798
[100]	valid_0's l1: 20.7216
[120]	valid_0's l1: 20.1679
[140]	valid_0's l1: 19.7634
[160]	valid_0's l1: 19.4859
[180]	valid_0's l1: 19.3079
[200]	valid_0's l1: 19.1281
[220]	valid_0's l1: 18.9691
[240]	valid_0's l1: 18.8532
[260]	valid_0's l1: 18.6765
[280]	valid_0's l1: 18.543
[300]	valid_0's l1: 18.4094
[320]	valid_0's l1: 18.2916
[340]	valid_0's l1: 18.1536
[360]	valid_0's l1: 18.0761
[380]	valid_0's l1: 17.9855
[400]	valid_0's l1: 17.8672
[420]	valid_0's l1: 17.8153
[440]	valid_0's l1: 17.721
[460]	valid_0's l1: 17.6234
[480]	valid_0's l1: 17.5553
[500]	valid_0's l1: 17.5068
[520]	valid_0's l1: 17.4334
[540]	valid_0's l1: 17.3612
[560]	valid_0's l1: 17.2835
[580]	valid_0's l1: 17.2484
[600]	valid_0's l1: 17.1989
[620]	valid_0's l1: 17.1409
[640]	valid_0's l1: 17.0973
[660]	valid_0's l1: 17.056
[680]	valid_0's l1: 16.9725
[700]	valid_0's l1: 16.9246
[720]	valid_0's l1: 16.8724


In [11]:
subname = '-quacking'
vername = '-engineering-4-2'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,220.106569
1,1_2024-06-04,174.033439
2,1_2024-06-05,166.057039
3,1_2024-06-06,186.434400
4,1_2024-06-07,216.314057
5,1_2024-06-08,146.198281
6,1_2024-06-09,145.518931
7,1_2024-06-10,198.708112
8,1_2024-06-11,152.256584
9,1_2024-06-12,177.870415


#### (3) + week_no만 추가로 카테고리 지정 

*다시 시작*

In [1]:
import sys
import os
import gc

import warnings
warnings.filterwarnings('ignore')

import random
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
import duckdb

import lightgbm as lgb
import xgboost as xgb
import catboost as cat

import shap

from sklearn.inspection import permutation_importance
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from sklearn import metrics

import matplotlib.pyplot as plt
from plotnine import * 
from mizani.breaks import date_breaks
from mizani.formatters import date_format

import duckdb
duckdb.query('PRAGMA disable_progress_bar;')


RANDOM_STATE = 1966
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(RANDOM_STATE)

class Shhh:
    # some of these models are still quite chatty even after disabling logging
    # we use this to swallow the printed output.
    # see: https://stackoverflow.com/questions/72346178/how-to-suppress-automatically-generated-output-from-a-python-code
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, "w")

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout
        sys.stderr = self._original_stderr
        
%matplotlib inline
%config InlineBackend.figure_format='retina'

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

BASEPATH = '/Users/djroz/Desktop/kaggle_study/rohlik-sales-v2-data/'

# borrowed from https://www.kaggle.com/code/samvelkoch/rohlik2-lama-v6-weighted

calendar = duckdb.query(f'''
    SELECT 
       t.* exclude(holiday_name),
       coalesce(t.holiday_name, '-') as holiday_name
    FROM 
        read_csv('{BASEPATH + 'calendar.csv'}') t
    ''').df()

from datetime import datetime
czech_holiday = [ 
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]
brno_holiday = [
    (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
    (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
]

budapest_holidays = []
munich_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

frank_holidays = [
    (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
    (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
]

def fill_loss_holidays(df_fill, warehouses, holidays):
    df = df_fill.copy()
    for item in holidays:
        dates, holiday_name = item
        generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
        for generated_date in generated_dates:
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
            df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
    return df

calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Brno_1'], holidays=brno_holiday)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Munich_1'], holidays=munich_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Frankfurt_1'], holidays=frank_holidays)
calendar = fill_loss_holidays(df_fill=calendar, warehouses=['Budapest_1'], holidays=budapest_holidays)

Frankfurt_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Frankfurt_1"')
Prague_2 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_2"')
Brno_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Brno_1"')
Munich_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Munich_1"')
Prague_3 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_3"')
Prague_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Prague_1"')
Budapest_1 = calendar.query('date >= "2020-08-01 00:00:00" and warehouse =="Budapest_1"')

def process_calendar(df):
    df = df.sort_values('date').reset_index(drop=True)
    df['next_holiday_date'] = df.loc[df['holiday'] == 1, 'date'].shift(-1)
    df['next_holiday_date'] = df['next_holiday_date'].bfill()
    df['days_to_holiday'] = (df['next_holiday_date'] - df['date']).dt.days
    df.drop(columns=['next_holiday_date'], inplace=True)
    df['next_shops_closed_date'] = df.loc[df['shops_closed'] == 1, 'date'].shift(-1)
    df['next_shops_closed_date'] = df['next_shops_closed_date'].bfill()
    df['days_to_shops_closed'] = (df['next_shops_closed_date'] - df['date']).dt.days
    df.drop(columns=['next_shops_closed_date'], inplace=True)
    df['day_after_closing'] = (
        (df['shops_closed'] == 0) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    
    df['long_weekend'] = (
        (df['shops_closed'] == 1) & (df['shops_closed'].shift(1) == 1)
    ).astype(int)
    df['school'] = df['winter_school_holidays'] + df['school_holidays']
    return df
dfs = ['Frankfurt_1', 'Prague_2', 'Brno_1', 'Munich_1', 'Prague_3', 'Prague_1', 'Budapest_1']
processed_dfs = [process_calendar(globals()[df]) for df in dfs]
calendar_extended = (
    pd.concat(processed_dfs)
    .sort_values('date')
    .drop(columns=['winter_school_holidays', 'school_holidays'])
    #.query('date <= "2024-06-02"')
    .reset_index(drop=True))

In [2]:
# read and join data

duckdb.query(f'''
    create or replace table sales as
    SELECT 
        s.*,
        'train' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_train.csv'}') s
    WHERE
        -- 52 rows have a null sales observation. 
        -- we just drop them.
        s.sales is not null
    UNION ALL BY NAME
    SELECT 
        s.*,
        'test' as rowtype
    FROM 
        read_csv('{BASEPATH + 'sales_test.csv'}') s
    ''')


duckdb.query(f'''
    create or replace table inventory as
    SELECT 
       t.* exclude(L1_category_name_en, L2_category_name_en, L3_category_name_en, L4_category_name_en),
       L1_category_name_en as l1_cat,
       L1_category_name_en || '-' ||  L2_category_name_en as l2_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en as l3_cat,
       L1_category_name_en || '-' ||  L2_category_name_en || '-' ||  L3_category_name_en || '-' ||  L3_category_name_en as l4_cat
    FROM 
        read_csv('{BASEPATH + 'inventory.csv'}') t
    ''')

duckdb.query(f'''
    create or replace table calendar as
    SELECT 
       *
    FROM 
        calendar_extended
    ''')

duckdb.query(f'''
    create or replace table test_weights as
    SELECT 
       *
    FROM 
        read_csv('{BASEPATH + 'test_weights.csv'}') t
    ''')

duckdb.query(f'''
    create or replace view all_data as
    SELECT 
        s.*,
        c.* exclude(warehouse, date),
        i.* exclude(warehouse, unique_id),
        w.* exclude(unique_id),
        -- group into fortnights
        -- The whole of test is a single fold
        date_diff('day', '2020-07-20', s.date) // 14 as dt_fold,
        date_diff('day', '2020-07-20', s.date) % 14  as dt_fold_day,
    FROM 
        sales s
        join calendar c on c.warehouse = s.warehouse and c.date = s.date
        join inventory i on i.warehouse = s.warehouse and i.unique_id = s.unique_id
        join test_weights w on w.unique_id = s.unique_id
''')

In [3]:
# add groupings for previous weeks:

groupings = ['l1_cat', 'l2_cat', 'l3_cat', 'l4_cat', 'name']
for grouping in groupings:
    # without warehouse
    duckdb.query(f'''
        create or replace table {grouping}_hist as
        select dt_fold, {grouping}, mean(sales) as sales
        from all_data
        where dt_fold > 0
        group by dt_fold, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')
    # with warehouse
    duckdb.query(f'''
        create or replace table {grouping}_whist as
        select dt_fold, warehouse, {grouping}, mean(sales) as sales, stddev(sales) as sales_sd,
        from all_data
        where dt_fold > 0
        group by dt_fold, warehouse, {grouping}
        having sum(sales) is not null
        order by dt_fold, {grouping}''')

# same day:
duckdb.query(f'''
    create or replace table name_whist_day as
    select dt_fold, dt_fold_day, warehouse, name, sum(sales) as sales
    from all_data
    where dt_fold > 0
    group by dt_fold, dt_fold_day, warehouse, name
    having sum(sales) is not null
    order by dt_fold, warehouse, name''')

In [4]:
duckdb.query(f'''
    create or replace table fe_data as
    select 
        a.*,
        -- sales of similar products in previous fortnight.
        -- overall
        coalesce(l1_cat_hist.sales, 0) as l1_cat_m1,
        coalesce(l2_cat_hist.sales, 0) as l2_cat_m1,
        coalesce(l3_cat_hist.sales, 0) as l3_cat_m1,
        coalesce(l4_cat_hist.sales, 0) as l4_cat_m1,
        coalesce(name_hist.sales, 0) as name_m1,
        -- by warehouse
        coalesce(l1_cat_whist.sales, 0) as l1_cat_wm1,
        coalesce(l2_cat_whist.sales, 0) as l2_cat_wm1,
        coalesce(l3_cat_whist.sales, 0) as l3_cat_wm1,
        coalesce(l4_cat_whist.sales, 0) as l4_cat_wm1,
        -- name & warehouse last 3
        coalesce(m1.sales, 0) as name_wm1,
        coalesce(m1.sales_sd, 0) as name_sd_wm1,
        coalesce(m2.sales, 0) as name_wm2,
        coalesce(m3.sales, 0) as name_wm3,
        coalesce(m4.sales, 0) as name_wm4,
        coalesce(m5.sales, 0) as name_wm5,
        (coalesce(m1.sales, 0) * 1
         + coalesce(m2.sales, 0) * 0.8
         + coalesce(m3.sales, 0) * 0.6
         + coalesce(m4.sales, 0) * 0.3
         + coalesce(m5.sales, 0) * 0.1 ) as name_w_weighted,
        coalesce(name_whist_day.sales, 0) as day_sales_m1,
        extract(dayofweek FROM date) AS day_of_week, 
        extract(month FROM date) AS month, 
        extract(day FROM date) AS day_of_month,
        extract(year FROM date) as year,
        extract(dayofyear FROM date) as day_of_year,
        extract(week FROM date) as week_no,
    from 
        all_data a
        -- overall all warehouses:
        left join l1_cat_hist 
            on l1_cat_hist.dt_fold + 1 = a.dt_fold 
                and l1_cat_hist.l1_cat = a.l1_cat
        left join l2_cat_hist 
            on l2_cat_hist.dt_fold + 1 = a.dt_fold 
                and l2_cat_hist.l2_cat = a.l2_cat
        left join l3_cat_hist 
            on l3_cat_hist.dt_fold + 1 = a.dt_fold 
                and l3_cat_hist.l3_cat = a.l3_cat
        left join l4_cat_hist 
            on l4_cat_hist.dt_fold + 1 = a.dt_fold 
                and l4_cat_hist.l4_cat = a.l4_cat
        left join name_hist 
            on name_hist.dt_fold + 1 = a.dt_fold 
                and name_hist.name = a.name
        -- by warehouse
        left join l1_cat_whist 
            on l1_cat_whist.dt_fold + 1 = a.dt_fold 
                and l1_cat_whist.warehouse = a.warehouse 
                and l1_cat_whist.l1_cat = a.l1_cat
        left join l2_cat_whist
            on l2_cat_whist.dt_fold + 1 = a.dt_fold 
                and l2_cat_whist.warehouse = a.warehouse 
                and l2_cat_whist.l2_cat = a.l2_cat
        left join l3_cat_whist 
            on l3_cat_whist.dt_fold + 1 = a.dt_fold 
                and l3_cat_whist.warehouse = a.warehouse 
                and l3_cat_whist.l3_cat = a.l3_cat
        left join l4_cat_whist 
            on l4_cat_whist.dt_fold + 1 = a.dt_fold 
                and l4_cat_whist.warehouse = a.warehouse 
                and l4_cat_whist.l4_cat = a.l4_cat
        left join name_whist as m1 
            on m1.dt_fold + 1 = a.dt_fold 
                and m1.warehouse = a.warehouse 
                and m1.name = a.name
        left join name_whist as m2 
            on m2.dt_fold + 2 = a.dt_fold 
                and m2.warehouse = a.warehouse 
                and m2.name = a.name
        left join name_whist as m3 
            on m3.dt_fold + 3 = a.dt_fold 
                and m3.warehouse = a.warehouse 
                and m3.name = a.name
        left join name_whist as m4 
            on m4.dt_fold + 4 = a.dt_fold 
                and m4.warehouse = a.warehouse 
                and m4.name = a.name
        left join name_whist as m5 
            on m5.dt_fold + 5 = a.dt_fold 
                and m5.warehouse = a.warehouse 
                and m5.name = a.name
        left join name_whist_day 
            on name_whist_day.dt_fold + 1 = a.dt_fold
                and name_whist_day.dt_fold_day = a.dt_fold_day
                and name_whist_day.warehouse = a.warehouse 
                and name_whist_day.name = a.name
    where
        a.dt_fold > 2
''')

In [ ]:
TARGET_NAME = 'sales'

cat_features =  [
    'warehouse',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    'day_of_week', 'month', 'year', 
    'week_no' # 추가
]

features = [
    'warehouse', 'total_orders',
    'sell_price_main', 'type_0_discount', 'type_1_discount',
    'type_2_discount', 'type_3_discount', 'type_4_discount',
    'type_5_discount', 'type_6_discount',  
    'holiday',
    'shops_closed', 
    'days_to_holiday',
    'days_to_shops_closed', 'day_after_closing', 'long_weekend', 'school',
    'name', 
    'l1_cat', 
    'l2_cat',
    'l3_cat', 
    'l4_cat',
    #'l1_cat_m1', 'l2_cat_m1', 'l3_cat_m1', 
    'l4_cat_m1', 'name_m1', 
    #'l1_cat_wm1', 'l2_cat_wm1', 'l3_cat_wm1', 
    'l4_cat_wm1', 
    'name_wm1', 'name_sd_wm1',
    'name_wm2', 'name_wm3', 'name_wm4', 'name_wm5', 'name_w_weighted',
    'day_sales_m1',
    'day_of_week', 'month', 'day_of_month', 'year',
    'day_of_year', 'sin_dayofyear', 'cos_dayofyear',
    'week_no',        
]

In [8]:
all_data = duckdb.query(f'''
    select * from fe_data''').df()

all_data[cat_features] = all_data[cat_features].astype('category')

all_data['sin_dayofyear']=np.sin(2 * np.pi * all_data['day_of_year'] / 365)
all_data['cos_dayofyear']=np.cos(2 * np.pi * all_data['day_of_year'] / 365)

periods = [7, 14, 21, 28, 56, 112, 224, 365]
def add_lag_sales(df_sales):
    for period in periods:
        df_sales[f"sales_lag_{period}"] = df_sales.groupby("unique_id")["sales"].shift(period)
    
    return df_sales

In [9]:
all_data = add_lag_sales(all_data)

ts = all_data.query('rowtype == "test"')
tr = all_data.query('rowtype == "train" and dt_fold < 97')
vl = all_data.query('rowtype == "train" and dt_fold >= 97')

del all_data
gc.collect()

ts.shape, tr.shape, vl.shape

((47021, 65), (3759180, 65), (190794, 65))

In [ ]:
# tr.info()

In [10]:
for fold in sorted(vl['dt_fold'].unique()):
    fl = vl.query('dt_fold == @fold')
    fold_metric = metrics.mean_absolute_error(fl[TARGET_NAME], fl['name_wm1'], sample_weight=fl['weight'])
    print(f'Out of sample fortnight: {fold} metric: {fold_metric:3.4f}')

Out of sample fortnight: 97 metric: 30.2051
Out of sample fortnight: 98 metric: 32.5799
Out of sample fortnight: 99 metric: 34.5443
Out of sample fortnight: 100 metric: 32.7728


In [11]:
def fit_model(cls, params, tr, vl, ts=None):
    global shap_values, shap_vl
    model = cls(**params)

    callbacks = [lgb.log_evaluation(period=20)]
    model.fit(
        X=tr[features], y = tr[TARGET_NAME], sample_weight=tr['weight'],
        eval_set=[(vl[features], vl[TARGET_NAME])],
        eval_sample_weight=[vl['weight']],
        callbacks=callbacks
    )
    
    vl_preds = np.clip((model.predict(vl[features])), 0, np.inf)
    ts_preds = None
    if ts is not None:
        ts_preds = np.clip((model.predict(ts[features])), 0, np.inf)
    
    if shap_values is None:
        # Calculate shap values on first model, first fold:
        if len(vl) > 50_000:
            shap_vl = vl.copy().sample(n=50_000)
        else:
            shap_vl = vl.copy()
        shap_values = shap.TreeExplainer(model).shap_values(shap_vl[features])
        pass
    
    return vl_preds, ts_preds

shap_values = None
shap_vl = None

params = {
 'learning_rate': 0.05,
 'max_depth': 10,
 'subsample': 0.7,
 'colsample_bytree': 0.7,
 'num_boost_round': 11000,
 'early_stopping_rounds': 500,
 'objective': 'regression',
 'metric': 'mae',
 'boosting_type': 'gbdt',
 'max_bin': 3000,
 'random_seed': 2112,
 'verbose':  -1}

vl_preds, ts_preds = fit_model(lgb.LGBMRegressor, params, tr, vl, ts)

[20]	valid_0's l1: 36.6501
[40]	valid_0's l1: 27.2172
[60]	valid_0's l1: 24.2789
[80]	valid_0's l1: 22.9414
[100]	valid_0's l1: 22.2616
[120]	valid_0's l1: 21.8462
[140]	valid_0's l1: 21.575
[160]	valid_0's l1: 21.3539
[180]	valid_0's l1: 21.1784
[200]	valid_0's l1: 21.0808
[220]	valid_0's l1: 20.9971
[240]	valid_0's l1: 20.9411
[260]	valid_0's l1: 20.8564
[280]	valid_0's l1: 20.7861
[300]	valid_0's l1: 20.7258
[320]	valid_0's l1: 20.7249
[340]	valid_0's l1: 20.6752
[360]	valid_0's l1: 20.6271
[380]	valid_0's l1: 20.5831
[400]	valid_0's l1: 20.5649
[420]	valid_0's l1: 20.536
[440]	valid_0's l1: 20.5234
[460]	valid_0's l1: 20.5071
[480]	valid_0's l1: 20.4872
[500]	valid_0's l1: 20.4856
[520]	valid_0's l1: 20.4542
[540]	valid_0's l1: 20.4398
[560]	valid_0's l1: 20.416
[580]	valid_0's l1: 20.4012
[600]	valid_0's l1: 20.3874
[620]	valid_0's l1: 20.3579
[640]	valid_0's l1: 20.35
[660]	valid_0's l1: 20.3528
[680]	valid_0's l1: 20.3458
[700]	valid_0's l1: 20.3438
[720]	valid_0's l1: 20.3269
[

In [12]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

[20]	valid_0's l1: 36.3959
[40]	valid_0's l1: 26.3581
[60]	valid_0's l1: 23.1558
[80]	valid_0's l1: 21.5752
[100]	valid_0's l1: 20.6661
[120]	valid_0's l1: 20.0955
[140]	valid_0's l1: 19.6835
[160]	valid_0's l1: 19.3736
[180]	valid_0's l1: 19.1948
[200]	valid_0's l1: 19.0058
[220]	valid_0's l1: 18.8463
[240]	valid_0's l1: 18.6973
[260]	valid_0's l1: 18.5386
[280]	valid_0's l1: 18.4088
[300]	valid_0's l1: 18.3049
[320]	valid_0's l1: 18.2024
[340]	valid_0's l1: 18.0739
[360]	valid_0's l1: 17.9735
[380]	valid_0's l1: 17.8913
[400]	valid_0's l1: 17.7885
[420]	valid_0's l1: 17.7194
[440]	valid_0's l1: 17.6093
[460]	valid_0's l1: 17.5358
[480]	valid_0's l1: 17.4662
[500]	valid_0's l1: 17.4308
[520]	valid_0's l1: 17.3634
[540]	valid_0's l1: 17.297
[560]	valid_0's l1: 17.2328
[580]	valid_0's l1: 17.1709
[600]	valid_0's l1: 17.0981
[620]	valid_0's l1: 17.0344
[640]	valid_0's l1: 17.0015
[660]	valid_0's l1: 16.9344
[680]	valid_0's l1: 16.8997
[700]	valid_0's l1: 16.8604
[720]	valid_0's l1: 16.83

In [13]:
subname = '-quacking'
vername = '-engineering-4-3'

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv(f'submission{subname}{vername}.csv', index=False))

pd.read_csv(f'submission{subname}{vername}.csv').head(100)

,id,sales_hat
0,1_2024-06-03,206.613550
1,1_2024-06-04,165.778260
2,1_2024-06-05,157.747371
3,1_2024-06-06,159.324930
4,1_2024-06-07,200.162092
5,1_2024-06-08,153.204405
6,1_2024-06-09,154.109716
7,1_2024-06-10,204.233165
8,1_2024-06-11,153.692855
9,1_2024-06-12,172.776496


### refit and submission code

In [ ]:
params_refit = params.copy()
del params_refit['early_stopping_rounds']
params_refit['num_boost_round'] = 7600

# don't trust the validation shown here:
_, ts_preds = fit_model(lgb.LGBMRegressor, params_refit, pd.concat([tr, vl]), vl, ts)

(ts
    .assign(**{
        'sales_hat': ts_preds,
        'id': lambda x: x['unique_id'].astype(str) + "_" + x['date'].astype(str)})
    .sort_values(['unique_id', 'date'])
    .filter(['id', 'sales_hat'])
    .to_csv('submission.csv', index=False))

pd.read_csv('submission.csv').head(100)